# 21

In [179]:
num_keypad = [['7','8','9'],
              ['4','5','6'],
              ['1','2','3'],
              ['X','0','A']]

num_position = (3,2) # row, col 

dir_keypad = [['X', '^', 'A'],
              ['<', 'v', '>']]

dir_position = (0,2) # row, col

In [77]:
def get_sequences(code, keypad, position):
    # Initialize the current position to the starting position
    current = position
   
    # Known paths: 
    result = ['']

    # Go character by character in the code
    for char in code: 
        # Find the position of the character on the keypad
        char_position = None

        for row in range(len(keypad)):
            for col in range(len(keypad[0])):
                if keypad[row][col] == char:
                    char_position = (row, col)

        # Find all paths from the current position to the character 
        paths = [('', current)] # (path, position)
        path_stubs = []

        while len(paths) > 0: 
            new_paths = []
            for path, position in paths:
                if position[0] < char_position[0]:
                    if keypad[position[0]+1][position[1]] != 'X':
                        new_paths.append((path+'v', (position[0]+1, position[1])))
                elif position[0] > char_position[0]:
                    if keypad[position[0]-1][position[1]] != 'X':
                        new_paths.append((path+'^', (position[0]-1, position[1])))

                if position[1] < char_position[1]:
                    if keypad[position[0]][position[1]+1] != 'X':
                        new_paths.append((path+'>', (position[0], position[1]+1)))
                elif position[1] > char_position[1]:
                    if keypad[position[0]][position[1]-1] != 'X':
                        new_paths.append((path+'<', (position[0], position[1]-1)))

                if position == char_position:
                    path = path + 'A'
                    path_stubs.append(path)
            paths = new_paths

        # print(f"char: {char}", path_stubs)
        # Update the current position 
        current = char_position

        # Add the shortest path to the result
        updated_result = []
        for path in result: 
            for stub in path_stubs:
                
                updated_result.append(path + stub)

        result = updated_result

    return result         

In [75]:
def first_sequence(code):
    return get_sequences(code, num_keypad, num_position)

def second_sequence(code):
    return get_sequences(code, dir_keypad, dir_position)


In [76]:
second_sequence('<^A')



['v<<A>^A>A', '<v<A>^A>A']

In [83]:
seqs =second_sequence('v<<A>^A>A')

In [84]:
for s in seqs:
     print(second_sequence(s))

['v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>^>AvA^Av<A^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>^>AvA^Av<A>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>^>AvA^A<vA^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>^>AvA^A<vA>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>>^AvA^Av<A^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>>^AvA^Av<A>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>>^AvA^A<vA^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A^>A<Av<A>>^AvA^A<vA>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>^>AvA^Av<A^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>^>AvA^Av<A>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>^>AvA^A<vA^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>^>AvA^A<vA>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>>^AvA^Av<A^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>>^AvA^Av<A>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>>^AvA^A<vA^>A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<Av>A^Av<A>^A<Av<A>>^AvA^A<vA>^A<A>A', 'v<A<A>^>Av<<A>^>AAvA^<

### Solution
Oh my, this is not elegant ... 

In [62]:
codes = []
with open("input.txt") as f: 
    for line in f: 
        codes.append(line.strip())
codes

['593A', '508A', '386A', '459A', '246A']

In [85]:
for s in first_sequence('379A'):
   print('----------------')
   
   print(s, len(second_sequence(s)[0])) # ^^<<, ^<<^, This: <<^^ | ^<^<, <^^<, <^<^
   print('---')
   for s2 in second_sequence(s):
        print(s2, len(second_sequence(s2)[0]))

----------------
^A^^<<A>>AvvvA 28
---
<A>A<AAv<AA>^>AvAA^Av<AAA^>A 70
<A>A<AAv<AA>^>AvAA^Av<AAA>^A 70
<A>A<AAv<AA>^>AvAA^A<vAAA^>A 70
<A>A<AAv<AA>^>AvAA^A<vAAA>^A 70
<A>A<AAv<AA>>^AvAA^Av<AAA^>A 68
<A>A<AAv<AA>>^AvAA^Av<AAA>^A 68
<A>A<AAv<AA>>^AvAA^A<vAAA^>A 68
<A>A<AAv<AA>>^AvAA^A<vAAA>^A 68
----------------
^A^<^<A>>AvvvA 32
---
<A>A<Av<A>^Av<A>^>AvAA^Av<AAA^>A 84
<A>A<Av<A>^Av<A>^>AvAA^Av<AAA>^A 84
<A>A<Av<A>^Av<A>^>AvAA^A<vAAA^>A 84
<A>A<Av<A>^Av<A>^>AvAA^A<vAAA>^A 84
<A>A<Av<A>^Av<A>>^AvAA^Av<AAA^>A 82
<A>A<Av<A>^Av<A>>^AvAA^Av<AAA>^A 82
<A>A<Av<A>^Av<A>>^AvAA^A<vAAA^>A 82
<A>A<Av<A>^Av<A>>^AvAA^A<vAAA>^A 82
----------------
^A^<<^A>>AvvvA 28
---
<A>A<Av<AA>^A>AvAA^Av<AAA^>A 70
<A>A<Av<AA>^A>AvAA^Av<AAA>^A 70
<A>A<Av<AA>^A>AvAA^A<vAAA^>A 70
<A>A<Av<AA>^A>AvAA^A<vAAA>^A 70
----------------
^A<^^<A>>AvvvA 32
---
<A>Av<<A>^AAv<A>^>AvAA^Av<AAA^>A 78
<A>Av<<A>^AAv<A>^>AvAA^Av<AAA>^A 78
<A>Av<<A>^AAv<A>^>AvAA^A<vAAA^>A 78
<A>Av<<A>^AAv<A>^>AvAA^A<vAAA>^A 78
<A>Av<<A>^AAv<A>>^AvAA^Av<AA

In [45]:
result = 0 

for code in codes: 
    second_sequences = []

    for sequence in first_sequence(code):
        #print(sequence)
        second_sequences.extend(second_sequence(sequence))

    second_sequences.sort(key=lambda x: len(x))
    

    third_sequences = []
    for sequence in second_sequences:
        #print(sequence)
        third_sequences.extend(second_sequence(sequence))

    third_sequences.sort(key=lambda x: len(x))

    # compute the complexity 
    complexity = len(third_sequences[0]) * int(code[:-1])

    result += complexity

result

157892

## Part 2

redo...

In [116]:
def find_dir_sequences(current, target, keypad = dir_keypad):
    # Find all paths from the current position to the character 
    path_stubs = []

    paths = [('', current)] 
    while len(paths) > 0: 
        new_paths = []
        for path, init_position in paths:
            if init_position[0] < target[0]:
                if keypad[init_position[0]+1][init_position[1]] != 'X':
                    new_paths.append((path+'v', (init_position[0]+1, init_position[1])))
            elif init_position[0] > target[0]:
                if keypad[init_position[0]-1][init_position[1]] != 'X':
                    new_paths.append((path+'^', (init_position[0]-1, init_position[1])))

            if init_position[1] < target[1]:
                if keypad[init_position[0]][init_position[1]+1] != 'X':
                    new_paths.append((path+'>', (init_position[0], init_position[1]+1)))
            elif init_position[1] > target[1]:
                if keypad[init_position[0]][init_position[1]-1] != 'X':
                    new_paths.append((path+'<', (init_position[0], init_position[1]-1)))

            if init_position == target:
                path = path + 'A'
                path_stubs.append(path)
        paths = new_paths

    return path_stubs

def precompute_dir_sequences(keypad = dir_keypad): 
    sequences = {}
    for init_row in range(len(keypad)):
        for init_col in range(len(keypad[0])):
            for target_row in range(len(keypad)):
                for target_col in range(len(keypad[0])):
                    sequences[(init_row, init_col), (target_row, target_col)] = find_dir_sequences((init_row, init_col), (target_row, target_col), keypad)
    return sequences

In [117]:
dir_keypad_sequences = precompute_dir_sequences()
num_keypad_sequences = precompute_dir_sequences(num_keypad)

In [151]:
dir_keypad_sequences 

{((0, 0), (0, 0)): ['A'],
 ((0, 0), (0, 1)): ['>A'],
 ((0, 0), (0, 2)): ['>>A'],
 ((0, 0), (1, 0)): ['vA'],
 ((0, 0), (1, 1)): ['v>A', '>vA'],
 ((0, 0), (1, 2)): ['v>>A', '>v>A', '>>vA'],
 ((0, 1), (0, 0)): [],
 ((0, 1), (0, 1)): ['A'],
 ((0, 1), (0, 2)): ['>A'],
 ((0, 1), (1, 0)): ['v<A'],
 ((0, 1), (1, 1)): ['vA'],
 ((0, 1), (1, 2)): ['v>A', '>vA'],
 ((0, 2), (0, 0)): [],
 ((0, 2), (0, 1)): ['<A'],
 ((0, 2), (0, 2)): ['A'],
 ((0, 2), (1, 0)): ['v<<A', '<v<A'],
 ((0, 2), (1, 1)): ['v<A', '<vA'],
 ((0, 2), (1, 2)): ['vA'],
 ((1, 0), (0, 0)): [],
 ((1, 0), (0, 1)): ['>^A'],
 ((1, 0), (0, 2)): ['>^>A', '>>^A'],
 ((1, 0), (1, 0)): ['A'],
 ((1, 0), (1, 1)): ['>A'],
 ((1, 0), (1, 2)): ['>>A'],
 ((1, 1), (0, 0)): [],
 ((1, 1), (0, 1)): ['^A'],
 ((1, 1), (0, 2)): ['^>A', '>^A'],
 ((1, 1), (1, 0)): ['<A'],
 ((1, 1), (1, 1)): ['A'],
 ((1, 1), (1, 2)): ['>A'],
 ((1, 2), (0, 0)): [],
 ((1, 2), (0, 1)): ['^<A', '<^A'],
 ((1, 2), (0, 2)): ['^A'],
 ((1, 2), (1, 0)): ['<<A'],
 ((1, 2), (1, 1)): ['<A'

In [152]:
optimized_sequences = {((0, 0), (0, 0)): ['A'],
 ((0, 0), (0, 1)): ['>A'],
 ((0, 0), (0, 2)): ['>>A'],
 ((0, 0), (1, 0)): ['vA'],
 ((0, 0), (1, 1)): ['v>A'],
 ((0, 0), (1, 2)): ['v>>A'],
 ((0, 1), (0, 0)): [],
 ((0, 1), (0, 1)): ['A'],
 ((0, 1), (0, 2)): ['>A'],
 ((0, 1), (1, 0)): ['v<A'],
 ((0, 1), (1, 1)): ['vA'],
 ((0, 1), (1, 2)): ['v>A'],
 ((0, 2), (0, 0)): [],
 ((0, 2), (0, 1)): ['<A'],
 ((0, 2), (0, 2)): ['A'],
 ((0, 2), (1, 0)): ['<v<A'],
 ((0, 2), (1, 1)): ['<vA'],
 ((0, 2), (1, 2)): ['vA'],
 ((1, 0), (0, 0)): [],
 ((1, 0), (0, 1)): ['>^A'],
 ((1, 0), (0, 2)): ['>^>A'],
 ((1, 0), (1, 0)): ['A'],
 ((1, 0), (1, 1)): ['>A'],
 ((1, 0), (1, 2)): ['>>A'],
 ((1, 1), (0, 0)): [],
 ((1, 1), (0, 1)): ['^A'],
 ((1, 1), (0, 2)): ['^>A'],
 ((1, 1), (1, 0)): ['<A'],
 ((1, 1), (1, 1)): ['A'],
 ((1, 1), (1, 2)): ['>A'],
 ((1, 2), (0, 0)): [],
 ((1, 2), (0, 1)): ['<^A'],
 ((1, 2), (0, 2)): ['^A'],
 ((1, 2), (1, 0)): ['<<A'],
 ((1, 2), (1, 1)): ['<A'],
 ((1, 2), (1, 2)): ['A']}

In [177]:
def get_sequences_memo(code, keypad, init_position):
    memo = num_keypad_sequences if keypad == num_keypad else optimized_sequences
    # Initialize the current position to the starting position
    current = init_position
   
    # Known paths: 
    result = ''

    # Go character by character in the code
    for char in code: 
        # Find the position of the character on the keypad
        char_position = None

        for row in range(len(keypad)):
            for col in range(len(keypad[0])):
                if keypad[row][col] == char:
                    char_position = (row, col)

       
        path_stubs = memo[(current, char_position)]
        current = char_position

        result = result + path_stubs[0]

    return result

In [173]:
first_sequences = first_sequence(code)
list(first_sequences)

['^^<AvvA^^^Avvv>A',
 '^^<AvvA^^^Avv>vA',
 '^^<AvvA^^^Av>vvA',
 '^^<AvvA^^^A>vvvA',
 '^<^AvvA^^^Avvv>A',
 '^<^AvvA^^^Avv>vA',
 '^<^AvvA^^^Av>vvA',
 '^<^AvvA^^^A>vvvA',
 '<^^AvvA^^^Avvv>A',
 '<^^AvvA^^^Avv>vA',
 '<^^AvvA^^^Av>vvA',
 '<^^AvvA^^^A>vvvA']

In [181]:
# Memoize on top of this? Wth
from tqdm import tqdm

result = 0

for code in codes:
    first_sequences = get_sequences_memo(code, num_keypad, num_position)
    current_sequences = first_sequences
    
    print("code: ", code)
    for _ in tqdm(range(15)):

        new_current_sequences = []
        for sequence in current_sequences:
            new_current_sequences.extend(get_sequences_memo(sequence, dir_keypad, dir_position))
       
        shortest_path_length = 10*len(sequence)

        for sequence in new_current_sequences:
            if len(sequence) < shortest_path_length:
                shortest_path_length = len(sequence)

        current_sequences = filter(lambda x: len(x) == shortest_path_length, new_current_sequences)

    result += shortest_path_length * int(code[:-1])

result

code:  593A


100%|██████████| 15/15 [00:08<00:00,  1.78it/s]


code:  508A


100%|██████████| 15/15 [00:12<00:00,  1.21it/s]


code:  386A


...

code:  459A


100%|██████████| 15/15 [00:11<00:00,  1.34it/s]


code:  246A


100%|██████████| 15/15 [00:08<00:00,  1.69it/s]


2192